In [1]:
# Data manipulation
import numpy as np
import pandas as pd

# Plotting
import plotly.express as px
import matplotlib.pyplot as plt
import streamlit as st

import datetime

#Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# import quant finance libraries
import yfinance as yf

In [15]:
stocks = ['aapl', 'msft']
start_date = datetime.date(2020, 1, 1)

In [9]:
def get_data(stocks, start_date, end_date):
            return yf.download(stocks, start = start_date, end = end_date)

In [10]:
stocks

['aapl', 'msft', 'team', 'googl', 'nvda']

In [17]:
start_date

datetime.date(2020, 1, 1)

In [18]:
df = get_data(stocks, start_date = "2022-01-01", end_date = "2022-11-30")       
df = df['Adj Close']

[*********************100%***********************]  2 of 2 completed

2 Failed downloads:
- AAPL: No data found for this date range, symbol may be delisted
- MSFT: No data found for this date range, symbol may be delisted
